In [230]:
#Importing Libraries
import numpy as np
import pandas as pd

In [231]:
dataSet = 'data4.xlsx'

In [232]:
#header = None states that there is no header row or else it would take first row of our data as header.
df = pd.read_excel(dataSet,sheet_name='Sheet1',header=None)
df = df.sample(frac=1).reset_index(drop=True)

In [233]:
valueArray = df.to_numpy()
np.random.shuffle(valueArray)
#Hold out cross validation 70 - 30
trainSize = int((7*valueArray.shape[0])/10)
trainData , testData = valueArray[0:trainSize,0:] , valueArray[trainSize:,0:]
X_train , X_test = trainData[0:,0:7] , testData[0:,0:7]
X_train = (X_train - X_train.mean())/X_train.std()
X_test = (X_test - X_test.mean())/X_test.std()
y_train , y_test = trainData[0:,7] , testData[0:,7]

In [234]:
y_train.shape , y_test.shape

((105,), (45,))

In [235]:
def prob(Xt,count,cov,mean,cov_det,cov_inverse):
    prob_x_y = (1/(np.power(2*np.pi,(7/2)))*np.power(cov_det,0.5))
   # print(prob_x_y)
    #print(-0.5*np.matmul(np.matmul((Xt-mean).T,cov_inverse),(Xt-mean)))
    prob_x_y = prob_x_y * np.exp(-0.5/1000*np.matmul(np.matmul((Xt-mean).T,cov_inverse),(Xt-mean)))
    return prob_x_y[0][0]

In [236]:
classes = [1,2,3]
count_classes = 3
y_test_pred = np.ndarray((y_test.shape[0],count_classes))
y_predicted = np.ndarray((y_test.shape[0]))

In [237]:
for i in range(y_test.shape[0]):
    for j in range(count_classes):
        temp = X_train[y_train==classes[j]]
        n = temp.shape[0]
        cov = np.cov(temp.T)
        cov_inv = np.linalg.inv(cov)
        cov_det = np.linalg.det(cov)
        mean = np.mean(temp,axis=0).reshape(-1,1)
        y_test_pred[i][j] = prob(X_test[i].reshape(-1,1),n,cov,mean,cov_det,cov_inv)
    y_predicted[i] = classes[np.argmax(y_test_pred[i])]

In [238]:
u11 = u12 = u13 = u21 = u22 = u23 = u31 = u32 = u33 = 0
for index in range(y_test.shape[0]):
    if y_test[index] == 1 :
        if y_predicted[index] == 1:
            u11 += 1
        elif y_predicted[index] == 2:
            u12 += 1
        else:
            u13 += 1
    elif y_test[index] == 2:
        if y_predicted[index] == 1:
            u21 += 1
        elif y_predicted[index] == 2:
            u22 += 1
        else:
            u23 += 1
    elif y_test[index] == 3:
        if y_predicted[index] == 1:
            u31 += 1
        elif y_predicted[index] == 2:
            u32 += 1
        else:
            u33 += 1

In [239]:
print("Confusion Matrix is :")
print(u11," ",u12," ",u13)
print(u21," ",u22," ",u23)
print(u31," ",u32," ",u33)

Confusion Matrix is :
15   0   0
14   0   0
16   0   0


In [240]:
IA_class1 = (float(u11)/float(u11 + u12 + u13))*100
IA_class2 = (float(u22)/float(u21 + u22 + u23))*100
IA_class3 = (float(u33)/float(u31 + u32 + u33))*100
Overall_Acc = (float(u11 + u22 + u33)/float(u11 + u12 + u13 + u21 + u22 + u23 + u31 + u32 + u33))*100

In [241]:
print("Individual Accuracy of class 1 is : ",IA_class1,"%")
print("Individual Accuracy of class 2 is : ",IA_class2,"%")
print("Individual Accuracy of class 3 is : ",IA_class3,"%")
print("Overall Accuracy is : ",Overall_Acc,"%")

Individual Accuracy of class 1 is :  100.0 %
Individual Accuracy of class 2 is :  0.0 %
Individual Accuracy of class 3 is :  0.0 %
Overall Accuracy is :  33.33333333333333 %
